# 10 - Export for App

## Overview
Export slim datasets for Streamlit app.

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
from f1ts import config, io_flat

## Load

In [ ]:
raw_dir = config.paths()['data_raw']
processed_dir = config.paths()['data_processed']
features_dir = config.paths()['data_features']

sessions = io_flat.read_csv(raw_dir / 'sessions.csv')
laps = io_flat.read_parquet(processed_dir / 'laps_processed.parquet')
stints = io_flat.read_parquet(processed_dir / 'stints.parquet')
strategies = io_flat.read_parquet(features_dir / 'strategy_decisions.parquet')

## Transform: Create App Exports

In [ ]:
app_export_dir = features_dir / 'app_export'
app_export_dir.mkdir(exist_ok=True)

# Race index
race_index = sessions[['session_key', 'circuit_name', 'date']].copy()
io_flat.write_csv(race_index, app_export_dir / 'race_index.csv')

# Export per-race files (sample for first race)
for session_key in sessions['session_key'].unique():
    session_laps = laps[laps['session_key'] == session_key]
    session_stints = stints[stints['session_key'] == session_key]
    
    if len(session_laps) > 0:
        # Slim down columns for app
        lap_cols = ['session_key', 'driver', 'lap', 'lap_time_ms', 'compound', 'stint_id', 'tyre_age_laps']
        available_cols = [c for c in lap_cols if c in session_laps.columns]
        io_flat.write_parquet(
            session_laps[available_cols],
            app_export_dir / f'{session_key}_laps.parquet'
        )

print(f'✓ Exported data for {len(sessions)} races')

## Save: Already saved above

## Repro Notes

- Exported slim datasets for Streamlit
- Created race index and per-race files